In [1]:
import pandas as pd
import numpy as np
import requests
import os
from requests import get
from bs4 import BeautifulSoup
import time
from time import sleep
from random import randint

In [2]:
#Lists we are going to write into
titles = []
years = []
run_time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
# Getting englight translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}

In [4]:
# Each page of IMDB contains 50 movie titles and we want 1000 movies
pages = np.arange(1,1001,50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [5]:
# sorting each of the URL of hte movies
for page in pages:
    # getting the content from each URL
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start='+str(page)+'&ref_=adv_nxt', headers = headers)
    soup = BeautifulSoup(page.text,'html.parser')
    
    # Aiming the part of the html we want to get hte info from
    movie_div = soup.find_all('div',class_='lister-item mode-advanced')
    
    # Controlling the loop's rate by pausing the execution of the loop for a specified amount of time
    #waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        
        #scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        #scraping the movie's Year
        year = container.h3.find('span',class_='lister-item-year').text
        years.append(year)
        
        # scraping hte movie's length
        length = container.find('span',class_='runtime').text if container.p.find('span',class_='runtime') else '-'
        run_time.append(length)
        
        # Scraping the rating
        rate = float(container.strong.text)
        imdb_ratings.append(rate)
        
        # Scraping the metascore
        meta = container.find('span',class_='metascore').text if container.find('span',class_='metascore') else '-'
        metascores.append(meta)
        
        #Scraping votes and gross earnings
        nv = container.find_all('span',attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        gross = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(gross)

In [6]:
# Creating a dataset out of the lists that we generated

Movies = pd.DataFrame({'Movies' : titles,
                       'Release_year' : years,
                       'Movie_length': run_time,
                       'Rating':imdb_ratings,
                       'Metascore':metascores,
                       'Votes':votes,
                       'Gross_earning':us_gross})
Movies.head()

,Movies,Release_year,Movie_length,Rating,Metascore,Votes,Gross_earning
0,Shang-Chi and the Legend of the Ten Rings,(2021),132 min,8.0,71,"40,479",-
1,Shershaah,(2021),135 min,8.8,-,"95,394",-
2,Once Upon a Time... In Hollywood,(2019),161 min,7.6,83,"631,469",$142.50M
3,The Shawshank Redemption,(1994),142 min,9.3,80,"2,456,113",$28.34M
4,Avengers: Endgame,(2019),181 min,8.4,78,"932,306",$858.37M


In [20]:
# Data cleaing, starting with release year
# Converting column Movies['Release_year'] into str first to clean it
Movies['Release_year'] = Movies['Release_year'].astype(str)
Movies['Release_year'] = Movies['Release_year'].str.extract('(\d+)').astype(int)
Movies['Movie_length'] = Movies['Movie_length'].str.extract('(\d+)').astype(int)
Movies['Metascore'] = Movies['Metascore'].str.extract('(\d+)')
# replacing '-' with nan and converting it to float
Movies['Metascore'] = pd.to_numeric(Movies['Metascore'], errors='coerce')
Movies['Votes'] = Movies['Votes'].str.replace(",","").astype(int)
